### Plot calibrated variables against original data 
To run this Notebook follow instructions at https://github.com/mbari-org/auv-python.

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src/data'));
if module_path not in sys.path:
    sys.path.append(module_path)
import xarray as xr
import hvplot.pandas
import ipywidgets as widgets
import pylab as plt
import pandas as pd
from logs2netcdfs import BASE_PATH, LOG_FILES

# Pick auv_name available from locally processed files
auv_picker = widgets.Dropdown(options=[f for f in sorted(os.listdir(BASE_PATH))
                                               if f not in ['.DS_Store', '.pytest_cache']])
display(auv_picker)

In [ ]:
# Pick auv mission on local system
auv_name = auv_picker.value
missions_dir = os.path.join(BASE_PATH, auv_name, 'missionnetcdfs')
mission_picker = widgets.Dropdown(options=[f for f in sorted(os.listdir(missions_dir))
                                                   if f != '.DS_Store'])
display(mission_picker)

In [ ]:
mission = mission_picker.value

# Read all the netCDF file data into in-memory xarray objects saved in a dictionary
netcdfs_dir = os.path.join(BASE_PATH, auv_name, 'missionnetcdfs', mission)
ncs = {}
for lf in LOG_FILES:
    nc_file = lf.replace('.log', '.nc')
    try:
        ncs[nc_file] = xr.open_dataset(os.path.join(netcdfs_dir, nc_file), decode_timedelta=False)
    except FileNotFoundError:
        pass    
cal_nc = f"{auv_name}_{mission}_cal.nc"
cal_nc4 = f"{auv_name}_{mission}_cal.nc4"
try:
    ncs[cal_nc] = xr.open_dataset(os.path.join(netcdfs_dir, cal_nc))
except FileNotFoundError:
    ncs[cal_nc] = xr.open_dataset(os.path.join(netcdfs_dir, cal_nc4))
# display(ncs[cal_nc])      # Uncomment to explore dataset

In [ ]:
%matplotlib ipympl

plt.figure(figsize=(12,3))
plt.plot(ncs[cal_nc]['depth_time'],
         ncs[cal_nc]['depth_filtdepth'], 'b-',
         ncs[cal_nc]['navigation_time'],
         ncs[cal_nc]['navigation_depth'], 'y:o',
         ncs['parosci.nc']['time'],
         ncs['parosci.nc']['depth'], '.',
         ncs[cal_nc]['seabird25p_time'],
         ncs[cal_nc]['seabird25p_depth'], 'c--',)
plt.legend(('depth_filtdepth', 'navigation_depth', 'parosci_depth', 'ctd1_depth'))
title = "Comparing Depth Variables"
plt.title(title)
plt.grid()

In [ ]:
# Print the variables in each data set
for nc_file, ds in ncs.items():
    print(f"{nc_file}: {[v for v in list(ds)]}")
    ##display(ds) # Uncomment to explore all the attributes

In [ ]:
# Pick the converted log to examine
ds_picker = widgets.Dropdown(options=ncs.keys())
display(ds_picker)

In [ ]:
# Pick the variable to plot
var_picker = widgets.Dropdown(options=list(ncs[ds_picker.value]))
display(var_picker)